In [1]:
import pytest
import copy
import numpy as np
from numpy.testing import assert_allclose

import os
from astropy import units as u
import sys, os, glob
sys.path.insert(0, "/home/bryanb/setigen")
import setigen as stg
import blimpy as bl

bshuf filter already loaded, skip it.
lz4 filter already loaded, skip it.
zfp filter already loaded, skip it.


In [2]:
sample_rate = 3e9
antenna = stg.voltage.Antenna(sample_rate=sample_rate, 
                              fch1=6*u.GHz,
                              ascending=True,
                              num_pols=2)

num_taps = 8
num_branches = 1024

digitizer = stg.voltage.RealQuantizer(target_fwhm=32,
                                      num_bits=8)

filterbank = stg.voltage.PolyphaseFilterbank(num_taps=num_taps, 
                                             num_branches=num_branches)

requantizer = stg.voltage.ComplexQuantizer(target_fwhm=32,
                                           num_bits=8)



In [3]:
num_taps = 8
num_branches = 1024
num_chans = 64
num_pols = 2
block_size = num_taps * num_chans * 2 * num_pols
rvb = stg.voltage.RawVoltageBackend(antenna,
                                    digitizer=digitizer,
                                    filterbank=filterbank,
                                    requantizer=requantizer,
                                    start_chan=0,
                                    num_chans=num_chans,
                                    block_size=block_size,
                                    blocks_per_file=128,
                                    num_subblocks=32)
antenna.x.add_noise(v_mean=0, 
                    v_std=1)
antenna.y.add_noise(v_mean=0, 
                    v_std=1)
antenna.x.add_constant_signal(f_start=6002.2e6, 
                              drift_rate=-2*u.Hz/u.s, 
                              level=0.002)
antenna.y.add_constant_signal(f_start=6002.2e6, 
                              drift_rate=-2*u.Hz/u.s, 
                              level=0.002,
                              phase=np.pi/2)

rvb.record(raw_file_stem='example_1block',
           num_blocks=1, 
           length_mode='num_blocks',
           header_dict={'HELLO': 'test_value',
                        'TELESCOP': 'GBT'},
           verbose=False)

wf_data = stg.voltage.get_waterfall_from_raw('example_1block.0000.raw',
                                             block_size=block_size,
                                             num_chans=num_chans,
                                             int_factor=1,
                                             fftlength=1)
print(wf_data.shape)

Blocks: 100%|██████████| 1/1 [00:00<00:00, 40.54it/s]

(8, 64)


In [6]:
np.std(wf_data)

523.8099729119477

In [20]:
header_dict = {}
with open('example_1block.0000.raw', "rb") as f:
    i = 1
    chunk = f.read(80)
    while True:
        key, item = chunk.decode().split('=')
        header_dict[key.strip()] = item.strip().strip("''")
        
        chunk = f.read(80)
        if f"{'END':<80}".encode() in chunk:
            break
        i += 1
header_dict

{'BACKEND': 'GUPPI   ',
 'TELESCOP': 'GBT     ',
 'OBSERVER': 'SETIGEN ',
 'PROJID': 'AGBT20B_999_22',
 'FRONTEND': 'RcvrArray18_26',
 'NRCVR': '2',
 'FD_POLN': 'CIRC    ',
 'BMAJ': '0.009263915095687008',
 'BMIN': '0.009263915095687008',
 'SRC_NAME': 'SYNTHETIC',
 'TRK_MODE': 'TRACK   ',
 'RA_STR': '04:41:45.7920',
 'RA': '70.4408',
 'DEC_STR': '+25:41:27.9600',
 'DEC': '25.6911',
 'LST': '83464',
 'AZ': '433.0963',
 'ZA': '69.1473',
 'DAQCTRL': 'start   ',
 'DAQPULSE': 'Tue Sep 22 00:24:27 2020',
 'DAQSTATE': 'record  ',
 'NBITS': '8',
 'OFFSET0': '0.0',
 'OFFSET1': '0.0',
 'OFFSET2': '0.0',
 'OFFSET3': '0.0',
 'BANKNAM': 'BLP00   ',
 'TFOLD': '0',
 'DS_FREQ': '1',
 'DS_TIME': '1',
 'FFTLEN': '512',
 'CHAN_BW': '2.9296875',
 'BANDNUM': '0',
 'NBIN': '0',
 'OBSNCHAN': '64',
 'SCALE0': '1.0',
 'SCALE1': '1.0',
 'DATAHOST': 'blr2-1-10-0.gb.nrao.edu',
 'SCALE3': '1.0',
 'NPOL': '4',
 'POL_TYPE': 'AABBCRCI',
 'BANKNUM': '0',
 'DATAPORT': '60000',
 'ONLY_I': '0',
 'CAL_DCYC': '0.5',
 'DIRE